In [1]:
import pandas as pd
from numpy.random import choice, randint
import uuid
from datetime import datetime, timezone, timedelta
from tqdm import tqdm
import concurrent.futures
from joblib import Parallel, delayed

In [2]:
# Формат представления данных
#
# logs = {
#     "timestamp": "DateTime",
#     "level": "TRACE/DEBUG/.../ERROR",
#     "microservice_id": "String",
#     "operation_type": "VIEW/BUY/CANCEL/REFUND",
#     "operation_id": "UUID",
#     "user_id": "UUID",
#     "message": "Arbitrary string"
# }
#
# metrics = {
#     "timestamp": "DateTime",
#     "microservice_id": "String",
#     "operation_type": "GET/POST or view,buy,cancel,refund",
#     "operation_id": "UUID",
#     "user_id": "UUID",
#     "value": "Arbitrary value"  #exec time, cpu/gpu consumption, retry count
# }
EVENT_TYPE = ['VIEW', 'TRANSACTION', 'CPU', 'RAM']
MICROSERVICE_ID = ['VIEW', 'BUY', 'CANCEL', 'REFUND']
LEVEL = ['TRACE', 'DEBUG', 'ERROR']
OPERATION_TYPE = MICROSERVICE_ID
OPERATION_TYPE.append('GET')
OPERATION_TYPE.append('POST')


In [3]:
simultaneous_users = 12
average_time_in_site = 6 #sec
potencial_users = 1000

potential_operation_id = 100
simultaneous_operation_id = 3

user_db = []
for user_id in tqdm(range(potencial_users)):
    user_db.append(str(uuid.uuid4()))


100%|██████████| 1000/1000 [00:00<00:00, 76928.65it/s]


In [4]:

# def common_metrics(start,simultaneous_users=simultaneous_users, average_time_in_site=average_time_in_site):
#     users = []
#     metrics = {}
#     dt_now=datetime.now().replace(tzinfo=timezone.utc).timestamp()
#     if  (dt_now - start) > average_time_in_site or not users:
#         users = choice(user_db, simultaneous_users)
#         start = datetime.now().replace(tzinfo=timezone.utc).timestamp()
#     metrics['timestamp'] = dt_now
#     metrics['microservice_id'] = choice(MICROSERVICE_ID)
#     metrics['operation_id'] = str(uuid.uuid4())
#     metrics['user_id'] = choice(users)
#     metrics['event_type'] = choice(EVENT_TYPE)
#
#     def generate_metrics_value():
#         return 1
#
#     metrics['value'] = generate_metrics_value()
#     return metrics

In [5]:
# запускать многопоточно
new = []
def common_metrics_per_one():
    user_metrics = []
    operation_ids = [str(uuid.uuid4()) for _ in range (3)]
    user = choice(user_db)
    for i in range(randint(1, 30)):
        metrics = {}
        dt_now=datetime.now().replace(tzinfo=timezone.utc).timestamp()
        metrics['timestamp'] = dt_now
        metrics['microservice_id'] = choice(MICROSERVICE_ID)
        metrics['operation_id'] = choice(operation_ids)
        metrics['user_id'] = user
        metrics['event_type'] = choice(EVENT_TYPE)

        def generate_metrics_value():
            return 1

        metrics['value'] = generate_metrics_value()
        user_metrics.append(metrics)
    return user_metrics

In [ ]:
results = Parallel(n_jobs=12)(delayed(common_metrics_per_one)(user) for user in users)

In [ ]:
import multiprocessing

def driver_func():
    PROCESSES = 5
    with multiprocessing.Pool(PROCESSES) as pool:
        users = choice(user_db, 50)
        results = [pool.apply_async(common_metrics_per_one, user) for user in users]

        for r in results:
            print('\t', r.get())
driver_func()

In [ ]:
import multiprocessing

def double(a):
    return a * 2

def driver_func():
    PROCESSES = 4
    with multiprocessing.Pool(PROCESSES) as pool:
        params = [(1, ), (2, ), (3, ), (4, )]
        results = [pool.apply_async(double, p) for p in params]

        for r in results:
            print('\t', r.get())

driver_func()

In [1]:
import pandas as pd
from numpy.random import choice, randint
import uuid
from datetime import datetime, timezone, timedelta
from tqdm import tqdm
import concurrent.futures
from joblib import Parallel, delayed
import multiprocessing
from multiprocessing import Pool
from types import FunctionType
import marshal


EVENT_TYPE = ['VIEW', 'TRANSACTION', 'CPU', 'RAM']
MICROSERVICE_ID = ['VIEW', 'BUY', 'CANCEL', 'REFUND']
LEVEL = ['TRACE', 'DEBUG', 'ERROR']
OPERATION_TYPE = MICROSERVICE_ID
OPERATION_TYPE.append('GET')
OPERATION_TYPE.append('POST')

simultaneous_users = 12
average_time_in_site = 6 #sec
potencial_users = 1000

potential_operation_id = 100
simultaneous_operation_id = 3

user_db = []
for user_id in tqdm(range(potencial_users)):
    user_db.append(str(uuid.uuid4()))

def common_metrics_per_one(user):
    user_metrics = []
    operation_ids = [str(uuid.uuid4()) for _ in range (3)]
    # user = choice(user_db)
    for i in range(randint(1, 30)):
        metrics = {}
        dt_now=datetime.now().replace(tzinfo=timezone.utc).timestamp()
        metrics['timestamp'] = dt_now
        metrics['microservice_id'] = choice(MICROSERVICE_ID)
        metrics['operation_id'] = choice(operation_ids)
        metrics['user_id'] = user
        metrics['event_type'] = choice(EVENT_TYPE)

        def generate_metrics_value():
            return 1

        metrics['value'] = generate_metrics_value()
        user_metrics.append(metrics)
    return user_metrics

def driver_func():
    PROCESSES = 5
    with multiprocessing.Pool(PROCESSES) as pool:
        users = choice(user_db, 50)
        results = [pool.apply_async(common_metrics_per_one, user) for user in users]
    return results
def _applicable(*args, **kwargs):
    name = kwargs['__pw_name']
    code = marshal.loads(kwargs['__pw_code'])
    gbls = globals() #gbls = marshal.loads(kwargs['__pw_gbls'])
    defs = marshal.loads(kwargs['__pw_defs'])
    clsr = marshal.loads(kwargs['__pw_clsr'])
    fdct = marshal.loads(kwargs['__pw_fdct'])
    func = FunctionType(code, gbls, name, defs, clsr)
    func.fdct = fdct
    del kwargs['__pw_name']
    del kwargs['__pw_code']
    del kwargs['__pw_defs']
    del kwargs['__pw_clsr']
    del kwargs['__pw_fdct']
    return func(*args, **kwargs)

def make_applicable(f, *args, **kwargs):
    if not isinstance(f, FunctionType): raise ValueError('argument must be a function')
    kwargs['__pw_name'] = f.__name__  # edited
    kwargs['__pw_code'] = marshal.dumps(f.__code__)   # edited
    kwargs['__pw_defs'] = marshal.dumps(f.__defaults__)  # edited
    kwargs['__pw_clsr'] = marshal.dumps(f.__closure__)  # edited
    kwargs['__pw_fdct'] = marshal.dumps(f.__dict__)   # edited
    return _applicable, args, kwargs

def _mappable(x):
    x,name,code,defs,clsr,fdct = x
    code = marshal.loads(code)
    gbls = globals() #gbls = marshal.loads(gbls)
    defs = marshal.loads(defs)
    clsr = marshal.loads(clsr)
    fdct = marshal.loads(fdct)
    func = FunctionType(code, gbls, name, defs, clsr)
    func.fdct = fdct
    return func(x)

def make_mappable(f, iterable):
    if not isinstance(f, FunctionType): raise ValueError('argument must be a function')
    name = f.__name__    # edited
    code = marshal.dumps(f.__code__)   # edited
    defs = marshal.dumps(f.__defaults__)  # edited
    clsr = marshal.dumps(f.__closure__)  # edited
    fdct = marshal.dumps(f.__dict__)  # edited
    return _mappable, ((i,name,code,defs,clsr,fdct) for i in iterable)

100%|██████████| 1000/1000 [00:00<00:00, 142843.17it/s]


In [7]:
if __name__ == "__main__":
    users = choice(user_db, 10)
    pool    = Pool(processes=2)
    results = [pool.apply_async(*make_applicable(common_metrics_per_one, user)) for user in users]
    data = [result.get(timeout=10) for result in results]

TimeoutError: 

In [5]:
data

[<bound method ApplyResult.get of <multiprocessing.pool.ApplyResult object at 0x0000020A28BCEA60>>,
 <bound method ApplyResult.get of <multiprocessing.pool.ApplyResult object at 0x0000020A28BCEBB0>>,
 <bound method ApplyResult.get of <multiprocessing.pool.ApplyResult object at 0x0000020A28BCECD0>>,
 <bound method ApplyResult.get of <multiprocessing.pool.ApplyResult object at 0x0000020A28BCEDF0>>,
 <bound method ApplyResult.get of <multiprocessing.pool.ApplyResult object at 0x0000020A4147B400>>,
 <bound method ApplyResult.get of <multiprocessing.pool.ApplyResult object at 0x0000020A4147B760>>,
 <bound method ApplyResult.get of <multiprocessing.pool.ApplyResult object at 0x0000020A4147B850>>,
 <bound method ApplyResult.get of <multiprocessing.pool.ApplyResult object at 0x0000020A4147BA30>>,
 <bound method ApplyResult.get of <multiprocessing.pool.ApplyResult object at 0x0000020A4147BBE0>>,
 <bound method ApplyResult.get of <multiprocessing.pool.ApplyResult object at 0x0000020A4147BDF0>>]